# RAG Fusion

You can also run this notebook online [at Noteable.io](https://app.noteable.io/published/d9902d51-c5e9-4d89-bcb1-f82521ab4497/rag_fusion).

This notebook shows off a LangChain JS port of [this Github repo](https://github.com/Raudaschl/rag-fusion) - all credit to the original author!

> RAG-Fusion, a search methodology that aims to bridge the gap between traditional search paradigms and the multifaceted dimensions of human queries. Inspired by the capabilities of Retrieval Augmented Generation (RAG), this project goes a step further by employing multiple query generation and Reciprocal Rank Fusion to re-rank search results.

## Setup

For this example we'll use an in memory store as our vectorstore/retriever, and some fake data. You can swap out the vectorstore for your [preferred LangChain.js option](https://js.langchain.com/docs/integrations/vectorstores) later.


In [1]:
// Deno.env.set("OPENAI_API_KEY", "");

import { OpenAIEmbeddings } from "npm:langchain@0.0.172/embeddings/openai";
import { MemoryVectorStore } from "npm:langchain@0.0.172/vectorstores/memory";

In [2]:
/** Define our fake data */
const allDocuments = [
  { id: "doc1", text: "Climate change and economic impact." },
  { id: "doc2", text: "Public health concerns due to climate change." },
  { id: "doc3", text: "Climate change: A social perspective." },
  { id: "doc4", text: "Technological solutions to climate change." },
  { id: "doc5", text: "Policy changes needed to combat climate change." },
  { id: "doc6", text: "Climate change and its impact on biodiversity." },
  { id: "doc7", text: "Climate change: The science and models." },
  { id: "doc8", text: "Global warming: A subset of climate change." },
  { id: "doc9", text: "How climate change affects daily weather." },
  { id: "doc10", text: "The history of climate change activism." },
];

In [3]:
/** Initialize our vector store with the fake data and OpenAI embeddings. */
const vectorStore = await MemoryVectorStore.fromTexts(
  allDocuments.map(({ text }) => text),
  allDocuments.map(({ id }) => ({ id })),
  new OpenAIEmbeddings()
);
/** Create the retriever */
const retriever = vectorStore.asRetriever();

## Define the Query Generator

We will now define a chain to do the query generation
This chain [pulls a prompt](https://smith.langchain.com/hub/langchain-ai/rag-fusion-query-generation) from the [LangChain Hub](https://smith.langchain.com/hub) that when provided a query, it tasks the model to generate multiple search queries related to the original. In our case, we're asking for 4 additional queries.

In [4]:
import { ChatOpenAI } from "npm:langchain@0.0.172/chat_models/openai";
import { pull } from "npm:langchain@0.0.172/hub";
import { StringOutputParser } from "npm:langchain@0.0.172/schema/output_parser";
import { RunnableLambda, RunnableSequence } from "npm:langchain@0.0.172/schema/runnable";

In [5]:
/** Define the chat model */
const model = new ChatOpenAI({
  modelName: "o4-mini",
  temperature: 0,
});

In [6]:
/** Pull a prompt from the hub */
const prompt = await pull("langchain-ai/rag-fusion-query-generation");
//  const prompt = ChatPromptTemplate.fromMessages([
//    ["system", "You are a helpful assistant that generates multiple search queries based on a single input query."],
//    ["user", "Generate multiple search queries related to: {original_query}"],
//    ["user", "OUTPUT (4 queries):"],
//  ]);

In [7]:
/** Define our chain for generating queries  */
const generateQueries = RunnableSequence.from([
  prompt,
  model,
  new StringOutputParser(),
  (output) => output.split("\n"),
]);

## Construct the Reciprocal Rank Fusion function
This function is used for combining the results of multiple search queries to produce a single ranked list of results. This is a common technique in information retrieval known as data fusion or result merging.

In [8]:
import { Document } from "npm:langchain@0.0.172/document";

In [9]:
const reciprocalRankFusion = (results: Document[][], k = 60) => {
  const fusedScores: Record<string, number> = {};
  for (const result of results) {
    // Assumes the docs are returned in sorted order of relevance
    result.forEach((item, index) => {
      const docString = item.pageContent;
      if (!(docString in fusedScores)) {
        fusedScores[docString] = 0;
      }
      fusedScores[docString] += 1 / (index + k);
    });
  }

  const rerankedResults = Object.entries(fusedScores)
    .sort((a, b) => b[1] - a[1])
    .map(
      ([doc, score]) => new Document({ pageContent: doc, metadata: { score } })
    );
  return rerankedResults;
};

## Define the full chain
Now we can put all our pieces together in one chain.
The chain performs the following steps:

1. Generate 4 search queries based on the original query
2. Perform lookups with the retriever for each generated query
3. Pass the results of the vector store lookup to the `reciprocalRankFusion` function

The `.map()` step on the retriever runs it on each query generated by `generateQueries`:

In [10]:
const chain = RunnableSequence.from([
  generateQueries,
  retriever.map(),
  reciprocalRankFusion,
]);

In [11]:
const originalQuery = "impact of climate change";

const result = await chain.invoke({
  original_query: originalQuery,
});

console.log(result);

[
  Document {
    pageContent: "Climate change and economic impact.",
    metadata: { score: 0.06558258417063283 }
  },
  Document {
    pageContent: "Climate change: A social perspective.",
    metadata: { score: 0.06400409626216078 }
  },
  Document {
    pageContent: "How climate change affects daily weather.",
    metadata: { score: 0.04787506400409626 }
  },
  Document {
    pageContent: "Climate change and its impact on biodiversity.",
    metadata: { score: 0.03306010928961749 }
  },
  Document {
    pageContent: "Public health concerns due to climate change.",
    metadata: { score: 0.016666666666666666 }
  },
  Document {
    pageContent: "Technological solutions to climate change.",
    metadata: { score: 0.016666666666666666 }
  },
  Document {
    pageContent: "Policy changes needed to combat climate change.",
    metadata: { score: 0.01639344262295082 }
  }
]
